In [14]:
import surprise 

print(surprise.__version__)

1.1.1


### Surprise 주요 모듈 소개

In [15]:
import pandas as pd
from surprise import Dataset 
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

ratings = pd.read_excel('./train_data.xlsx')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('./train_data_noh.csv', index=False, header=False)
# rating -> output으로

In [16]:
from surprise import Reader
#user ingredient cuisine
reader = Reader(line_format='user rating item', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./train_data_noh.csv',reader=reader)
# user item rating timestamp -> user cuisine 

In [17]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)


RMSE: 110.5015


110.50154676172951

### 교차 검증(Cross Validation)과 하이퍼 파라미터 튜닝

In [18]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_excel('./train_data.xlsx') # reading data in pandas df
reader = Reader(line_format='user rating item', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./train_data_noh.csv',reader=reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    102.6295110.4061103.2428110.1301112.4868107.77914.0420  
MAE (testset)     86.0513 95.7179 87.5897 91.6154 99.7949 92.1538 5.0882  
Fit time          0.01    0.01    0.01    0.01    0.01    0.01    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([102.62953041, 110.40612673, 103.24280567, 110.13010953,
        112.48680834]),
 'test_mae': array([86.05128205, 95.71794872, 87.58974359, 91.61538462, 99.79487179]),
 'fit_time': (0.009947061538696289,
  0.00997304916381836,
  0.009973287582397461,
  0.009973526000976562,
  0.008975744247436523),
 'test_time': (0.0, 0.0, 0.0, 0.0, 0.0009975433349609375)}

In [19]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

107.78159032539288
{'n_epochs': 20, 'n_factors': 50}


### Surprise 를 이용한 개인화 영화 추천 시스템 구축

In [20]:
# 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생합니다.
reader = Reader(line_format='user rating item', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./train_data_noh.csv',reader=reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)


AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

In [21]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user rating item', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds=Dataset.load_from_file('./train_data_noh.csv',reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()


In [22]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)


In [23]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
cuisines = pd.read_excel('./cuisine.xlsx')
# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
cuisinesIds = ratings[ratings['user']==1]['cuisineid']
if cuisinesIds[cuisinesIds==42].count() == 0:
    print('사용자 아이디 1는 영화 아이디 42의 평점 없음')
else:
    print('done')

print(cuisines[cuisines['cuisineid']==42])


done
    cuisineid  user          cuisine ingredients ingredients.1 ingredients.2  \
42         42     1  Egg fried rice4         egg          rice   green onion   

   ingredients.3 ingredients.4 ingredients.5 ingredients.6  ... ingredients.9  \
42    sesame oil  oyster sauce        pepper           NaN  ...           NaN   

   ingredients.10 ingredients.11 ingredients.12 ingredients.13 ingredients.14  \
42            NaN            NaN            NaN            NaN            NaN   

   ingredients.15 ingredients.16 ingredients.17 ingredients.18  
42            NaN            NaN            NaN            NaN  

[1 rows x 22 columns]


In [24]:
uid = str(1)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)


user: 1          item: 42         r_ui = None   est = 5.00   {'was_impossible': False}


In [27]:
def get_unseen_surprise(ratings, cuisine, user):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_cuisines = ratings[ratings['user']== user]['cuisineid'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_cuisines = cuisines['cuisineid'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_cuisines= [cuisine for cuisine in total_cuisines if cuisine not in seen_cuisines]
    print('평점 매긴 음식수:',len(seen_cuisines), '추천대상 음식수:',len(unseen_cuisines), \
          '전체 음식수:',len(total_cuisines))
    
    return unseen_cuisines

unseen_cuisines = get_unseen_surprise(ratings, cuisines, 9)

평점 매긴 음식수: 0 추천대상 음식수: 195 전체 음식수: 195


In [30]:
def recomm_movie_by_surprise(algo, user, unseen_cuisines, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(user), str(cuisineid)) for cuisineid in unseen_cuisines]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_cuisine_ids = [ int(pred.iid) for pred in top_predictions]
    top_cuisine_rating = [ pred.est for pred in top_predictions]
    top_cuisine_titles = cuisines[cuisines.cuisine.isin(top_cuisine_ids)]['cuisineid']
    top_cuisine_preds = [ (id, title, rating) for id, title, rating in zip(top_cuisine_ids, top_cuisine_titles, top_cuisine_rating)]
    
    return top_cuisine_preds

unseen_cuisines = get_unseen_surprise(ratings, cuisines, 9)
top_cuisine_preds = recomm_movie_by_surprise(algo, 9, unseen_cuisines, top_n=10)
print('##### Top-10 추천 음식 리스트 #####')

for top_cuisines in top_cuisine_preds:
    print(top_cuisines[1], ":", top_cuisines[2])

평점 매긴 음식수: 0 추천대상 음식수: 195 전체 음식수: 195
##### Top-10 추천 음식 리스트 #####
